In [1]:
import pandas as pd
import json
from common import wbi_config, login, sparql_prefixes
from wikibaseintegrator import WikibaseIntegrator, datatypes, wbi_helpers
from wikibaseintegrator.models import References, Reference, Qualifiers

In [2]:
import requests

For each ncbi taxon ID, retrieve nucleotide accessions for that taxon, and filter for 16S rRNA sequences (keyword search for "16S", "SSU", "small subunit" in title).

If there is only one SSU sequence available in the NCBI database for this taxon ID, then it must be the representative SSU rRNA sequence.

In [3]:
ppsdb_query = """
SELECT DISTINCT ?item ?ncbi WHERE {
  ?item ppt:P11 ?ncbi.
  FILTER NOT EXISTS { ?item ppt:P34 ?ssu. }
}
"""

In [4]:
elink = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi"

esummary = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"

In [5]:
ppsdb_recs = wbi_helpers.execute_sparql_query(query=ppsdb_query, prefix=sparql_prefixes)

In [6]:
qid2repseq = {}

In [10]:
for rec in ppsdb_recs['results']['bindings']:
    qid = rec['item']['value'].split("/")[-1]
    taxid = rec['ncbi']['value']
    print(qid, taxid)
    nuccore_query = {
        "dbfrom" : "taxonomy",
        "db" : "nucleotide",
        "id" : taxid,
        "idtype" : "acc",
        "retmode" : "json",
    }
    try:
        get_out = requests.get(elink, params=nuccore_query)
    except:
        print("elink query returned exception, skipping")
        continue
    if not get_out.ok:
        print("elink query not ok")
        continue
    try:
        ids = [i['links'] for i in json.loads(get_out.text)['linksets'][0]['linksetdbs'] if i['dbto'] == 'nuccore' and i['linkname'] == 'taxonomy_nuccore'][0]
    except:
        print(f"no links for {taxid}, skipping")
        continue
    print(f"nuccore sequences found: {str(len(ids))}")
    if len(ids) > 200:
        print("over 200 sequences retrieved, skipping")
        continue
    summary_query = {
        "db" : "nucleotide",
        "id" : ",".join(ids),
        "retmode" : "json"
    }
    try:
        summary_out = requests.get(esummary, params=summary_query)
    except:
        print("esummary query returned exception, skipping")
        continue
    if not summary_out.ok:
        print("esummary query not ok")
        continue
    uids = json.loads(summary_out.text)['result']['uids']
    ssu_accs = []
    for i in uids:
        rec = json.loads(summary_out.text)['result'][i]
        tests = ['16S' in rec['title'], 'SSU' in rec['title'], '18S' in rec['title'], 'small subunit' in rec['title'].lower(), 'small-subunit' in rec['title'].lower()]
        if any(tests):
            ssu_accs.append((rec['title'], rec['accessionversion']))
    if len(ssu_accs) == 1:
        print(ssu_accs)
        qid2repseq[qid] = ssu_accs[0]

Q7 1905158
nuccore sequences found: 9
Q8 2126337
nuccore sequences found: 3641
over 200 sequences retrieved, skipping
Q16 219837
nuccore sequences found: 1
[('Geleia fossata small subunit ribosomal RNA gene, partial sequence', 'AY187925.1')]
Q23 1981150
nuccore sequences found: 541
over 200 sequences retrieved, skipping
Q24 1981151
nuccore sequences found: 222
over 200 sequences retrieved, skipping
Q30 2126338
nuccore sequences found: 707
over 200 sequences retrieved, skipping
Q31 1905156
nuccore sequences found: 22
Q32 2126562
nuccore sequences found: 602
over 200 sequences retrieved, skipping
Q33 2126340
nuccore sequences found: 3099
over 200 sequences retrieved, skipping
Q34 2126342
nuccore sequences found: 1166
over 200 sequences retrieved, skipping
Q35 2126335
nuccore sequences found: 2905
over 200 sequences retrieved, skipping
Q36 2138164
nuccore sequences found: 639
over 200 sequences retrieved, skipping
Q37 2126332
nuccore sequences found: 1816
over 200 sequences retrieved, ski

In [11]:
len(qid2repseq)

137

In [33]:
# with open("qid2repseq.json", "w") as fh:
#     json.dump(qid2repseq, fh)

In [3]:
# with open("qid2repseq.json", "r") as fh:
#     qid2repseq = json.load(fh)

In [6]:
wbi = WikibaseIntegrator(login=login)

In [8]:
for qid in qid2repseq:
    acc = qid2repseq[qid][1].split(".")[0]
    print(qid, acc)
    item = wbi.item.get(qid)
    item.claims.add(datatypes.String(prop_nr="P34", value=acc))
    item.write(summary="Link to representative SSU rRNA sequence by NCBI taxon ID")

Q16 AY187925
Q62 MW203126
Q63 MK543441
Q92 MK547175
Q118 AF107209
Q120 AF107210
Q122 U37762
Q130 AB073554
Q132 AB900796
Q136 U57696
Q138 LC086369
Q146 AY598719
Q151 KF697197
Q152 KF697195
Q163 LC025959
Q164 LC025974
Q166 KF705035
Q167 FJ766469
Q169 JF694281
Q170 AF177427
Q171 AF352390
Q172 AF352388
Q173 AF352387
Q174 AF352389
Q175 AF352391
Q176 AF132134
Q43 LT621912
Q61 AB713411
Q81 KT023596
Q107 AY364636
Q177 AF132137
Q179 AF132136
Q183 AF366580
Q185 AF366583
Q193 AM412761
Q197 AY741401
Q198 EF492067
Q203 KX156800
Q204 KX156801
Q205 KX156802
Q211 KR607499
Q216 LC025958
Q217 LC025975
Q199 OR532590
Q226 JF488667
Q227 JF488770
Q243 MK027054
Q244 AF193247
Q252 KM033243
Q255 KM033245
Q257 KM033232
Q258 KM033236
Q259 KM033234
Q260 KM033240
Q261 KM033235
Q262 KM033237
Q263 KM033256
Q264 KM033262
Q265 KM033247
Q266 KM033261
Q278 AB326375
Q296 AJ583377
Q300 DQ855405
Q304 AB458854
Q312 FN377761
Q313 FN377762
Q314 FN377763
Q315 FN377764
Q316 FN377765
Q317 FN377766
Q302 OL895181
Q335 AY512589
Q33